In [1]:
import numpy as np
import pandas as pd
import napari
import tifffile
import skimage as ski
import scipy.ndimage as ndi
import glob
import plotly.express as px
import cellpose.models as models
import matplotlib.pyplot as plt
import cv2
import dask

In [2]:
viewer = napari.Viewer()

In [3]:
def backsub_2D(inp, radius=60):
    filterSize =(radius, radius)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,
                                    filterSize)
    blurred = cv2.GaussianBlur(inp, (5, 5), 0)
    tophat_img = cv2.morphologyEx(blurred,
                                cv2.MORPH_TOPHAT,
                                kernel)
    rtn = inp.astype(np.single) - (blurred-tophat_img)
    rtn = np.clip(rtn, 0, np.inf)
    return rtn

In [4]:
def backsub_3D(inp, radius=60):
    process = [dask.delayed(backsub_2D)(inp[i], radius) for i in range(inp.shape[0])]
    rtn = np.stack(dask.compute(*process))

In [5]:
img = tifffile.imread('files/C-hela-cells.tif')
img.shape

(512, 672, 3)

In [6]:
viewer.layers.clear()
viewer.add_image(img, name=['lysosomes', 'mitocondria', 'nucleii'], colormap=['red', 'green', 'blue'], channel_axis=2)

[<Image layer 'lysosomes' at 0x1c797120dc0>,
 <Image layer 'mitocondria' at 0x1c79710e770>,
 <Image layer 'nucleii' at 0x1c7971b49a0>]

Standard pipeline:  gassian blur, subtract background, threshold

In [7]:
lyso = img[:,:,0].astype(float)
mitos = img[:,:,1].astype(float)
dapi = img[:,:,2].astype(float)

In [8]:
#background_subtracted = dapi - ski.restoration.rolling_ball(dapi, radius=120)
background_subtracted = backsub_2D(dapi, radius=200)
lyso = backsub_2D(lyso, radius=20)
mitos = backsub_2D(mitos, radius=20)
viewer.add_image(background_subtracted, name='background_subtracted', colormap='gray', blending='additive')

<Image layer 'background_subtracted' at 0x1c8090c79a0>

In [9]:
blurred = ndi.gaussian_filter(background_subtracted, 10)
viewer.add_image(blurred, name='blurred', colormap='gray', blending='additive')

<Image layer 'blurred' at 0x1c8091de560>

In [10]:
thresholded = blurred > 300
viewer.add_image(thresholded, name='thresholded', colormap='gray', blending='additive')

<Image layer 'thresholded' at 0x1c79710e050>

In [11]:
labels = ndi.label(thresholded)[0]

In [12]:
viewer.add_labels(labels)

<Labels layer 'labels' at 0x1c7ffe7ba00>

In [13]:
results = pd.DataFrame(ski.measure.regionprops_table(labels, mitos, properties=('label', 'area', 'centroid', 'mean_intensity')))
results

,label,area,centroid-0,centroid-1,mean_intensity
0,1,14407.0,179.571042,463.460609,65.856298
1,2,15429.0,171.777821,298.290881,66.723717
2,3,14932.0,279.711626,140.072998,73.606385
3,4,14258.0,408.910296,333.062141,82.264534


In [14]:
viewer.add_image(mitos, blending='additive')

<Image layer 'mitos' at 0x1c800596d70>

In [15]:
intensities_img = ski.util.map_array(labels, results['label'].values, results['mean_intensity'].values)

In [16]:
viewer.add_image(intensities_img, name='intensities', colormap='gray', blending='additive')

<Image layer 'intensities' at 0x1c800986590>

In [17]:
smoothed_mitos = ndi.gaussian_filter(mitos, 10)
viewer.add_image(smoothed_mitos, name='smoothed_mitos', colormap='gray', blending='additive')

<Image layer 'smoothed_mitos' at 0x1c800597e20>

In [18]:
smoothed_lyso = ndi.gaussian_filter(img[:,:,0], 10)
viewer.add_image(smoothed_lyso, name='smoothed_lyso', colormap='gray', blending='additive')

<Image layer 'smoothed_lyso' at 0x1c800d8ffa0>

In [19]:
masked_lyso = smoothed_lyso>350
viewer.add_image(masked_lyso, name='masked_lyso', colormap='gray', blending='additive')

<Image layer 'masked_lyso' at 0x1c809c30f40>

In [20]:
edt = ndi.distance_transform_edt(masked_lyso)
viewer.add_image(edt, name='edt', colormap='gray', blending='additive')

<Image layer 'edt' at 0x1c80d686a10>

In [21]:
watershedded = ski.segmentation.watershed(-edt, labels, mask=masked_lyso)
viewer.add_labels(watershedded, name='watershedded', blending='additive')

<Labels layer 'watershedded' at 0x1c80db7d6c0>

In [22]:
results = pd.DataFrame(ski.measure.regionprops_table(watershedded, mitos, properties=('label', 'area', 'centroid', 'mean_intensity')))
lyso_results = pd.DataFrame(ski.measure.regionprops_table(watershedded, lyso, properties=('label', 'area', 'centroid', 'mean_intensity')))

results['lyso_mean_intensity'] = lyso_results['mean_intensity']

In [23]:
results

,label,area,centroid-0,centroid-1,mean_intensity,lyso_mean_intensity
0,1,44235.0,188.152933,499.062462,101.636958,71.476131
1,2,50295.0,131.969520,319.039407,108.780831,54.511495
2,3,42868.0,243.536437,136.338947,99.017375,73.016421
3,4,38548.0,397.226497,366.374883,106.814139,94.192731


In [24]:
intensity_img = ski.util.map_array(watershedded, results['label'].values, results['lyso_mean_intensity'].values)
viewer.add_image(intensity_img, name='intensity_img', colormap='gray', blending='additive')

<Image layer 'intensity_img' at 0x1c80dc4c9a0>

# Find maxima